In [ ]:
import requests
import json
from csv import reader 
import os
import sys
from bs4 import BeautifulSoup
import time
import random
from lxml import etree
import os
import re
import datetime
# # from selenium import webdriver


In [ ]:
nowTime = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

In [ ]:
# year = ['2014','2015','2016','2017','2018','2019','2020','2021','2022']
domain = 'https://www.caranddriver.com'
# headers = {
#         'Accept': '*/*',
#         'Accept-Encoding': 'gzip, deflate, br',
#         'Host':'www.kbb.com',
#         'Referer':'https://www.kbb.com/car-reviews/',
#         'Content-Type':'application/json',
#         'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
#         }

In [ ]:
make=[
# 'Acura',
# 'AlfaRomeo',
# 'AstonMartin',
# 'Audi',
# 'Bentley',
# 'BMW',
# 'Bugatti',
# 'Buick',
# 'Cadillac',
# 'Chevrolet',
# 'Chrysler',
# 'Dodge',
# 'Ferrari',
'Fiat'#,
# 'Ford',
# 'Genesis',
# 'GMC',
# 'Honda',
# 'Hyundai',
# 'Infiniti',
# 'Jaguar',# 无年份
# 'Jeep',
# 'Kia',
# 'Lamborghini',
# 'LandRover',
# 'Lexus',
# 'Lincoln',
# 'Lotus',
# 'Maserati',
# 'Mazda',
# 'McLaren',
# 'Mercedes-AMG',
# 'Mercedes-Benz',
# 'Mercedes-Maybach',
# 'Mini',
# 'Mitsubishi',
# 'Nissan',
# 'Polestar',
# 'Porsche',
# 'Ram', # 无年份
# 'Rolls-Royce',
# 'Scion',
# 'Smart',
# 'Subaru',
# 'Suzuki',
# 'Tesla',
# 'Toyota',
# 'Volkswagen',
# 'Volvo'
]

In [ ]:
# get url for Specs & Rate
for m in make:
    url_main=domain+'/'+m.lower()
    # print(url_main)
    response = requests.get(url=url_main)
    soup=soup= BeautifulSoup(response.text, "lxml")
    href=soup.select('a[class="vehicle-item-title item-title"]')
    vehicle={}
    for u in href:
        url_each=domain+u.get('href')
        # print(url_each)
        response = requests.get(url=url_each)
        soup= BeautifulSoup(response.text, "lxml")
        name=soup.select('h1[class="css-i4j13t e10ise8i2"]')[0].text
        try:
            vehicle[name]={
            'year':re.findall('(^\d{4})',name)[0],
            'make':re.findall('[^ ]+',name)[1],
            'model':' '.join(re.findall('[^ ]+',name)[2:]),
            'kbb_url':'https://www.kbb.com'+'/'+re.findall('[^ ]+',name)[1].lower()+'/'+' '.join(re.findall('[^ ]+',name)[2:]).replace(' ','-')+'/'+re.findall('(^\d{4})',name)[0]+'/consumer-reviews',
            'cnd_url':domain+soup.select('#main-content > header > div.css-16uv341.endngjl0 > div.css-70qvj9.e17a0kax2 > div > a[title="Specs"]')[0].attrs['href']
            }
            # vehicle[name]['cnd_url']=domain+soup.select('#main-content > header > div.css-16uv341.endngjl0 > div.css-70qvj9.e17a0kax2 > div > a[title="Specs"]')[0].attrs['href']
        except IndexError:
            pass
    with open(m+'.json', "w") as dump_f:
        json.dump(vehicle, dump_f)

In [ ]:
for v in vehicle:
    print(vehicle[i]['cnd_url'])
    response = requests.get(url=vehicle[v]['cnd_url'])
    soup= BeautifulSoup(response.text, "lxml")
    specs={}
    for i in soup.select('div[class="css-qgjj1l e17ofjz23"]'):
        # print(i.select('button[class="css-17t01md e17ofjz22"]')[0].text)
        specs[i.select('button[class="css-17t01md e17ofjz22"]')[0].text]={}
        list=[]
        for j in i.select('div[class="css-1ajawdl e2zahha0"]'):
            try:
                specs[i.select('button[class="css-17t01md e17ofjz22"]')[0].text][j.contents[0].text]=j.contents[1].text
                # print(j.contents[0].text,j.contents[1].text)
            except IndexError:
                list.append(j.contents[0].text)
                print(j.contents[0].text)
                specs[i.select('button[class="css-17t01md e17ofjz22"]')[0].text]=list
    response = requests.get(url=vehicle[v]['kbb_url'])
    soup= BeautifulSoup(response.text, "lxml")
    try:
        rate={}
        star={}
        for i in soup.select('div[class="eov6l8h1 css-4dxonz ehh6yb86"]'):
            star[i.select('span[class="css-ihxznl ehh6yb82"]')[0].text]=i.select('span[class="css-1c6thby ehh6yb83"]')[0].text
        mark={}
        for i in soup.select('div[class="css-12jjwzq"]'):
            mark[i.select('div[class="css-1rttn8x"]')[0].text]=i.select('div[class="css-sf59yt"]')[0].text
            # print(i.select('div[class="css-1rttn8x"]')[0].text,i.select('div[class="css-sf59yt"]')[0].text)
        rate={
        'overall':soup.select('div[class="css-wti69m"]')[0].text,
        'count':re.findall('.*\([^\)\(\d]*(\d+)[^\)\(\d]*\).*',soup.select('div[class="css-14z3b1p"]')[0].text)[0],
        'recby': soup.select('span[class="css-19pqkoc e1agtnah1"]')[0].text,
        'star':star,
        'mark':mark
        }
    except IndexError:
        pass

In [ ]:
# test for get specs
url='https://www.caranddriver.com/ford/mustang-2022/specs'
response = requests.get(url=url)
soup= BeautifulSoup(response.text, "lxml")

specs={}
for i in soup.select('div[class="css-qgjj1l e17ofjz23"]'):
    print(i.select('button[class="css-17t01md e17ofjz22"]')[0].text)
    specs[i.select('button[class="css-17t01md e17ofjz22"]')[0].text]={}
    list=[]
    for j in i.select('div[class="css-1ajawdl e2zahha0"]'):
        try:
            specs[i.select('button[class="css-17t01md e17ofjz22"]')[0].text][j.contents[0].text]=j.contents[1].text
            # print(j.contents[0].text,j.contents[1].text)
        except IndexError:
            list.append(j.contents[0].text)
            print(j.contents[0].text)
            specs[i.select('button[class="css-17t01md e17ofjz22"]')[0].text]=list

In [ ]:
# test for get rate
url='https://www.kbb.com/fiat/500/2018/consumer-reviews/'
response = requests.get(url=url)
soup= BeautifulSoup(response.text, "lxml")
# html = etree.HTML(response.text)
# html.xpath()
try:
    rate={}
    star={}
    for i in soup.select('div[class="eov6l8h1 css-4dxonz ehh6yb86"]'):
        star[i.select('span[class="css-ihxznl ehh6yb82"]')[0].text]=i.select('span[class="css-1c6thby ehh6yb83"]')[0].text
    mark={}
    for i in soup.select('div[class="css-12jjwzq"]'):
        mark[i.select('div[class="css-1rttn8x"]')[0].text]=i.select('div[class="css-sf59yt"]')[0].text
        # print(i.select('div[class="css-1rttn8x"]')[0].text,i.select('div[class="css-sf59yt"]')[0].text)
    rate={
    'overall':soup.select('div[class="css-wti69m"]')[0].text,
    'count':re.findall('.*\([^\)\(\d]*(\d+)[^\)\(\d]*\).*',soup.select('div[class="css-14z3b1p"]')[0].text)[0],
    'recby': soup.select('span[class="css-19pqkoc e1agtnah1"]')[0].text,
    'star':star,
    'mark':mark
    }
except IndexError:
    pass

In [ ]:
#get each model from car&driver
soup.select('select[id="styleSelect"] option[value!="0"]:not([selected=""])')
str='https://www.kbb.com/fiat/500x/2022/consumer-reviews'
re.findall('^(.*[\\\/])',str) #获取/后所有
# re.findall('[^/]+(?!.*/)',str)#获取/前所有

In [ ]:
str='(2022222 reviews)'
re.findall('.*\([^\)\(\d]*(\d+)[^\)\(\d]*\).*',str)
re.findall('.*\([^\)\(\d]*(\d+)[^\)\(\d]*\).*',str)